# Graded Lab: Agentic Workflows

In this lab, you will build an agentic system that generates a short research report through planning, external tool usage, and feedback integration. Your workflow will involve:

### Agents

* **Planning Agent / Writer**: Creates an outline and coordinates tasks.
* **Research Agent**: Gathers external information using tools like Arxiv, Tavily, and Wikipedia.
* **Editor Agent**: Reflects on the report and provides suggestions for improvement.

---
<a name='submission'></a>

<h4 style="color:green; font-weight:bold;">TIPS FOR SUCCESSFUL GRADING OF YOUR ASSIGNMENT:</h4>

* All cells are frozen except for the ones where you need to write your solution code or when explicitly mentioned you can interact with it.

* In each exercise cell, look for comments `### START CODE HERE ###` and `### END CODE HERE ###`. These show you where to write the solution code. **Do not add or change any code that is outside these comments**.

* You can add new cells to experiment but these will be omitted by the grader, so don't rely on newly created cells to host your solution code, use the provided places for this.

* Avoid using global variables unless you absolutely have to. The grader tests your code in an isolated environment without running all cells from the top. As a result, global variables may be unavailable when scoring your submission. Global variables that are meant to be used will be defined in UPPERCASE.

* To submit your notebook for grading, first save it by clicking the 💾 icon on the top left of the page and then click on the <span style="background-color: red; color: white; padding: 3px 5px; font-size: 16px; border-radius: 5px;">Submit assignment</span> button on the top right of the page.
---


### Research Tools

By importing `research_tools`, you gain access to several search utilities:

- `research_tools.arxiv_search_tool(query)` → search academic papers from **arXiv**  

  *Example:* `research_tools.arxiv_search_tool("neural networks for climate modeling")`

- `research_tools.tavily_search_tool(query)` → perform web searches with the **Tavily API**  

  *Example:* `research_tools.tavily_search_tool("latest trends in sunglasses fashion")`

- `research_tools.wikipedia_search_tool(query)` → retrieve summaries from **Wikipedia**  

  *Example:* `research_tools.wikipedia_search_tool("Ensemble Kalman Filter")`

Run the cell below to make them available.

In [1]:
# =========================
# Imports
# =========================

# --- Standard library 
from datetime import datetime
import re
import json
import ast


# --- Third-party ---
from IPython.display import Markdown, display
from aisuite import Client

# --- Local / project ---
import research_tools

In [2]:
import unittests

### Initialize client

Create a shared client instance for upcoming calls.

In [5]:
CLIENT = Client()

## Exercise 1: planner_agent

### Objective
Correctly set up a call to a language model (LLM) to generate a research plan.

### Instructions

1. **Focus Areas**:
   - Ensure `CLIENT.chat.completions.create` is correctly configured.
   - Pass the `model` and `messages` parameters correctly:
     - **Model**: Use `"openai:o4-mini"` by default.
     - **Messages**: Set with `{"role": "user", "content": user_prompt}`.
     - **Temperature**: Fixed at 1 for creative outputs.

### Notes

- The prompt is pre-defined and guides the LLM on task requirements.
- Only return a formatted list of steps — no extra text.

Focus on the LLM call setup to complete the task.

In [6]:
# GRADED FUNCTION: planner_agent

def planner_agent(topic: str, model: str = "openai:o4-mini") -> list[str]:
    """
    Generates a plan as a Python list of steps (strings) for a research workflow.

    Args:
        topic (str): Research topic to investigate.
        model (str): Language model to use.

    Returns:
        List[str]: A list of executable step strings.
    """

    
    # Build the user prompt
    user_prompt = f"""
    You are a planning agent responsible for organizing a research workflow with multiple intelligent agents.

    🧠 Available agents:
    - A research agent who can search the web, Wikipedia, and arXiv.
    - A writer agent who can draft research summaries.
    - An editor agent who can reflect and revise the drafts.

    🎯 Your job is to write a clear, step-by-step research plan **as a valid Python list**, where each step is a string.
    Each step should be atomic, executable, and must rely only on the capabilities of the above agents.

    🚫 DO NOT include irrelevant tasks like "create CSV", "set up a repo", "install packages", etc.
    ✅ DO include real research-related tasks (e.g., search, summarize, draft, revise).
    ✅ DO assume tool use is available.
    ✅ DO NOT include explanation text — return ONLY the Python list.
    ✅ The final step should be to generate a Markdown document containing the complete research report.

    Topic: "{topic}"
    """

    # Add the user prompt to the messages list
    messages = [{"role": "user", "content": user_prompt}]

    ### START CODE HERE ###

    # Call the LLM
    response = CLIENT.chat.completions.create( 
        # Pass in the model
        model=model,
        # Define the messages. Remember this is meant to be a user prompt!
        messages=messages,
        # Keep responses creative
        temperature=1, 
    )

    ### END CODE HERE ###

    # Extract message from response
    steps_str = response.choices[0].message.content.strip()

    # Parse steps
    steps = ast.literal_eval(steps_str)

    return steps

In [7]:
# Test your code!
unittests.test_planner_agent(planner_agent)

 All tests passed!


## Exercise 2: research_agent

### Objective
Set up a call to a language model (LLM) to perform a research task using various tools.

### Instructions

**Focus Areas**:

- **Creating a Custom Prompt**:
  - **Define the Role**: Clearly specify the role, such as "research assistant."
  - **List Available Tools** (as strings inside the prompt, not the actual functions):
    - Use `arxiv_tool` to find academic papers.
    - Use `tavily_tool` for general web searches.
    - Use `wikipedia_tool` for accessing encyclopedic knowledge.
  - **Specify the Task**: Include a placeholder in your prompt for defining the specific task that needs to be accomplished.
  - **Include Date Information**: Add a placeholder for the current date or time to provide context.

- **Creating Messages Dict**:
  - Ensure the `messages` are correctly set with `{"role": "user", "content": prompt}`.

- **Creating Tools List**:
  - Create a list of tools for use, such as `research_tools.arxiv_search_tool`, `research_tools.tavily_search_tool`, and `research_tools.wikipedia_search_tool`.

- **Correctly Setting the Call to the LLM**:
  - Pass the `model`, `messages`, and `tools` parameters accurately.
  - Set `tool_choice` to `"auto"` for automatic tool selection.
  - Limit interactions with `max_turns=6`.

### Notes

- The function provides pre-coded blocks where you need to replace placeholder values.
- The approach allows the LLM to use tools dynamically based on the task.

Focus on accurately setting the messages, tools, and LLM call parameters to complete the task.

In [12]:
# GRADED FUNCTION: research_agent

def research_agent(task: str, model: str = "openai:gpt-4o", return_messages: bool = False):
    """
    Executes a research task using tools via aisuite (no manual loop).
    Returns either the assistant text, or (text, messages) if return_messages=True.
    """
    print("==================================")  
    print("🔍 Research Agent")                 
    print("==================================")

    current_time = datetime.now().strftime('%Y-%m-%d')
    
    ### START CODE HERE ###

    # Create a customizable prompt by defining the role (e.g., "research assistant"),
    # listing tools (arxiv_tool, tavily_tool, wikipedia_tool) for various searches,
    # specifying the task with a placeholder, and including a current_time placeholder.
    prompt = f"""
You are a highly capable AI research assistant. 
Today's date is {current_time}.

You have access to the following tools:
- **arxiv_tool** → to locate and summarize academic research papers and preprints from arXiv.
- **tavily_tool** → to search the general web for up-to-date or non-academic information.
- **wikipedia_tool** → to look up encyclopedic and background knowledge.

Your task:
{task}

Guidelines:
- Use the tools when needed to gather reliable, cited information.
- Summarize findings clearly and concisely.
- Include references or URLs for every key claim.
- Write your final output in an academic and structured manner, with clear sections (e.g., Introduction, Findings, Conclusion).
- Do not fabricate citations.
"""
    
    # Create the messages dict to pass to the LLM. Remember this is a user prompt!
    messages = [{"role": "user", "content": prompt}]

    # Save all of your available tools in the tools list. These can be found in the research_tools module.
    # You can identify each tool in your list like this: 
    # research_tools.<name_of_tool>, where <name_of_tool> is replaced with the function name of the tool.
    tools = [research_tools.arxiv_search_tool, research_tools.tavily_search_tool, research_tools.wikipedia_search_tool]
    
    # Call the model with tools enabled
    response = CLIENT.chat.completions.create(  
        # Set the model
        model=model,
        # Pass in the messages. You already defined this!
        messages=messages,
        # Pass in the tools list. You already defined this!
        tools=tools,
        # Set the LLM to automatically choose the tools
        tool_choice="automatic",
        # Set the max turns to 6
        max_turns=6
    )  
    
    ### END CODE HERE ###

    content = response.choices[0].message.content
    print("✅ Output:\n", content)

    
    return (content, messages) if return_messages else content  

In [13]:
# Test your code!
unittests.test_research_agent(research_agent)

🔍 Research Agent
✅ Output:
 ### Introduction
In the contemporary landscape of scientific inquiry, key domains such as artificial intelligence (AI), climate change, and quantum computing remain at the forefront of research and development. This report summarizes recent findings and insights from academic papers, reliable internet sources, and encyclopedic entries.

### Findings

#### 1. Artificial Intelligence
Recent research highlighted by arXiv offers significant insights into the governance and ethical considerations surrounding AI. A prominent study, "The Governance of Physical Artificial Intelligence" by Yingbo Li et al. (2023), addresses the complexities involved in managing AI's integration into society, emphasizing the need for responsible governance frameworks ([arXiv link](http://arxiv.org/abs/2304.02924v1)). Additional research by Takanobu Mizuta (2020) explores the unexpected outcomes of AI in financial markets, particularly its capacity to engage in market manipulation thro

## Exercise 3: writer_agent

### Objective
Set up a call to a language model (LLM) for executing writing tasks like drafting, expanding, or summarizing text.

### Instructions

1. **Focus Areas**:
   - **System Prompt**:
     - Define `system_prompt` to assign the LLM the role of a writing agent focused on generating academic or technical content.
   - **System and User Messages**:
     - Create `system_msg` using `{"role": "system", "content": system_prompt}`.
     - Create `user_msg` using `{"role": "user", "content": task}`.
   - **Messages List**:
     - Combine `system_msg` and `user_msg` into a `messages` list.

### Notes

- The function is designed to produce well-structured text by setting the correct prompts.
- Temperature is set to 1.0 to allow for creative variance in the writing outputs.

Ensure the system prompt and messages are defined properly to achieve a structured output from the LLM.

In [14]:
# GRADED FUNCTION: writer_agent
def writer_agent(task: str, model: str = "openai:gpt-4o") -> str: # @REPLACE def writer_agent(task: str, model: str = None) -> str:
    """
    Executes writing tasks, such as drafting, expanding, or summarizing text.
    """
    print("==================================")
    print("✍️ Writer Agent")
    print("==================================")

    ### START CODE HERE ###
    
    # Create the system prompt.
    # This should assign the LLM the role of a writing agent specialized in generating well-structured academic or technical content
    system_prompt = (
        "You are a professional academic and technical writing assistant. "
        "Your goal is to produce well-structured, clear, and coherent text "
        "for scholarly, scientific, or professional contexts. "
        "Write in a formal tone, ensure logical flow, and use precise language. "
        "When expanding or summarizing text, maintain factual accuracy and avoid repetition."
    )

    # Define the system msg by using the system_prompt and assigning the role of system
    system_msg = {"role": "system", "content": system_prompt}

    # Define the user msg. In this case the user prompt should be the task passed to the function
    user_msg = {"role": "user", "content": task}

    # Add both system and user messages to the messages list
    messages = [system_msg, user_msg]
    
    ### END CODE HERE ###

    response = CLIENT.chat.completions.create(
        model=model, 
        messages=messages,
        temperature=1.0
    )

    return response.choices[0].message.content

In [15]:
# Test your code!
unittests.test_writer_agent(writer_agent)

✍️ Writer Agent
 All tests passed!


## Exercise 4: editor_agent

### Objective
Configure a call to a language model (LLM) to perform editorial tasks such as reflecting, critiquing, or revising drafts.

### Instructions

1. **Focus Areas**:
   - **System Prompt**:
     - Define `system_prompt` to assign the LLM the role of an editor agent whose task is to reflect on, critique, or improve drafts.
   - **System and User Messages**:
     - Create `system_msg` using `{"role": "system", "content": system_prompt}`.
     - Create `user_msg` using `{"role": "user", "content": task}`.
   - **Messages List**:
     - Combine `system_msg` and `user_msg` into a `messages` list.

### Notes

- The editor agent is tailored for enhancing the quality of text by setting an appropriate role and task in the prompts.
- Temperature is set to 0.7, balancing creativity and coherence in editorial outputs.

Ensure the system prompt and messages are accurately set up to perform effective editorial tasks with the LLM.

In [16]:
# GRADED FUNCTION: editor_agent
def editor_agent(task: str, model: str = "openai:gpt-4o") -> str:
    """
    Executes editorial tasks such as reflection, critique, or revision.
    """
    print("==================================")
    print("🧠 Editor Agent")
    print("==================================")
    
    ### START CODE HERE ###

    # Create the system prompt.
    # This should assign the LLM the role of an editor agent specialized in reflecting on, critiquing, or improving existing drafts.
    system_prompt = (
        "You are a professional editor and writing reviewer. "
        "Your task is to reflect on, critique, and improve existing drafts. "
        "Provide structured feedback, identify strengths and weaknesses, "
        "and suggest or implement improvements in clarity, coherence, and style. "
        "Focus on enhancing the quality of the text while maintaining its original intent."
    )
    
    # Define the system msg by using the system_prompt and assigning the role of system
    system_msg = {"role": "system", "content": system_prompt}
    
    # Define the user msg. In this case the user prompt should be the task passed to the function
    user_msg = {"role": "user", "content": task}
    
    # Add both system and user messages to the messages list
    messages = [system_msg, user_msg]
    
    ### END CODE HERE ###
    
    response = CLIENT.chat.completions.create(
        model=model, 
        messages=messages,
        temperature=0.7 
    )
    
    return response.choices[0].message.content

In [17]:
# Test your code!
unittests.test_editor_agent(editor_agent)

🧠 Editor Agent
 All tests passed!


### 🎯 The Executor Agent

The `executor_agent` manages the workflow by executing each step of a given plan. It:

1. Decides **which agent** (`research_agent`, `writer_agent`, or `editor_agent`) should handle the step.
2. Builds context from the outputs of previous steps.
3. Sends the enriched task to the selected agent.
4. Collects and stores the results in a shared history.

👉 **Do not implement or modify this function.** It is already provided as the orchestration component of the multi-agent pipeline.

Notice that `planner_agent` might return a long list of steps. Because of this, the maximum number of steps is set to a maximum of 4 to keep running time reasonable.

In [18]:
agent_registry = {
    "research_agent": research_agent,
    "editor_agent": editor_agent,
    "writer_agent": writer_agent,
}

def clean_json_block(raw: str) -> str:
    """
    Clean the contents of a JSON block that may come wrapped with Markdown backticks.
    """
    raw = raw.strip()
    if raw.startswith("```"):
        raw = re.sub(r"^```(?:json)?\n?", "", raw)
        raw = re.sub(r"\n?```$", "", raw)
    return raw.strip()

In [19]:
def executor_agent(topic, model: str = "openai:gpt-4o", limit_steps: bool = True):

    plan_steps = planner_agent(topic)
    max_steps = 4

    if limit_steps:
        plan_steps = plan_steps[:min(len(plan_steps), max_steps)]
    
    history = []

    print("==================================")
    print("🎯 Editor Agent")
    print("==================================")

    for i, step in enumerate(plan_steps):

        agent_decision_prompt = f"""
        You are an execution manager for a multi-agent research team.

        Given the following instruction, identify which agent should perform it and extract the clean task.

        Return only a valid JSON object with two keys:
        - "agent": one of ["research_agent", "editor_agent", "writer_agent"]
        - "task": a string with the instruction that the agent should follow

        Only respond with a valid JSON object. Do not include explanations or markdown formatting.

        Instruction: "{step}"
        """
        response = CLIENT.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": agent_decision_prompt}],
            temperature=0,
        )

        raw_content = response.choices[0].message.content
        cleaned_json = clean_json_block(raw_content)
        agent_info = json.loads(cleaned_json)

        agent_name = agent_info["agent"]
        task = agent_info["task"]

        context = "\n".join([
            f"Step {j+1} executed by {a}:\n{r}" 
            for j, (s, a, r) in enumerate(history)
        ])
        enriched_task = f"""
        You are {agent_name}.

        Here is the context of what has been done so far:
        {context}

        Your next task is:
        {task}
        """

        print(f"\n🛠️ Executing with agent: `{agent_name}` on task: {task}")

        if agent_name in agent_registry:
            output = agent_registry[agent_name](enriched_task)
            history.append((step, agent_name, output))
        else:
            output = f"⚠️ Unknown agent: {agent_name}"
            history.append((step, agent_name, output))

        print(f"✅ Output:\n{output}")

    return history

In [20]:
# If you want to see the full workflow without limiting the number of steps. Set limit_steps to False
# Keep in mind this could take more than 10 minutes to complete
executor_history = executor_agent("The ensemble Kalman filter for time series forecasting", limit_steps=True)

md = executor_history[-1][-1].strip("`")  
display(Markdown(md))

🎯 Editor Agent

🛠️ Executing with agent: `research_agent` on task: Search academic databases and arXiv for papers on 'ensemble Kalman filter' and 'time series forecasting', retrieving the top relevant articles
🔍 Research Agent
✅ Output:
 ## Introduction

This research overview aims to explore the application of the ensemble Kalman filter and methodologies in time series forecasting. Although there was an issue accessing specific academic papers on the ensemble Kalman filter from the arXiv database, the exploration did yield substantial information about advances in time series forecasting.

## Findings

### Time Series Forecasting

1. **Inter Time Series Sales Forecasting (Gahirwal, 2013):**  
   This paper highlights how combining forecasts from different models can enhance the forecasting quality in time series analysis. The paper emphasizes decomposing a series into trend, seasonality, and irregular components for efficient forecasting using methods such as ARIMA and Holt-Winter. Th


🛠️ Executing with agent: `research_agent` on task: Survey recent applications of the ensemble Kalman filter in meteorology, finance, and engineering time series forecasting
🔍 Research Agent
✅ Output:
 # Recent Applications of the Ensemble Kalman Filter in Various Domains

## Introduction

The Ensemble Kalman Filter (EnKF) remains a significant tool for state estimation in high-dimensional systems, offering invaluable applications across meteorology, finance, and engineering. This review surveys recent research to highlight the diverse applications of EnKF in these domains, leveraging its strengths in data assimilation and forecasting.

## Findings

### Meteorology

1. **EnKF in High-Dimensional Meteorological Problems:**
   - Morzfeld, Hodyss, and Snyder (2015) explore the reliability of EnKF as a data assimilation tool for meteorological applications, emphasizing its utility despite potential collapses in certain high-dimensional problems. The concept of localizing particle filters i

# Comparative Performance of the Ensemble Kalman Filter: A Comprehensive Overview

## Introduction

This review aims to synthesize and summarize recent comparative studies and performance metrics of the Ensemble Kalman Filter (EnKF) across different domains. The review draws on a selection of academic papers and other resources to evaluate the EnKF's efficiency in state estimation and its applications across various complex systems.

## Findings

### Meteorology and Environmental Sciences

- **Kalman Filter-based Heuristic Ensemble (KFHE) Approach**:
  A novel perspective on multi-class ensemble classification was introduced, utilizing the Kalman filter for state estimation in ensemble learning. This approach, tested on 30 datasets, demonstrates its robustness in handling class label noise and outperforms traditional ensemble methods in classification tasks ([Pakrashi & Namee, 2018](http://arxiv.org/pdf/1807.11429v3)).

- **Pilot Point Ensemble Kalman Filter (PP-EnKF)**:
  The PP-EnKF is evaluated as a superior method for parameter estimation in geostatistical applications, outperforming the classical EnKF by reducing spurious correlations significantly in small ensemble sizes. This version also aligns closer with large-ensemble references in terms of ensemble variance estimation ([Keller et al., 2021](http://arxiv.org/pdf/2108.02164v1)).

- **Comparison of EnKF Variants**:
  Research comparing multiple EnKF variants, including the classical, damped, and iterative versions, indicates varying performance across scenarios. The evaluated parameters, such as root mean square errors (RMSE) and data assimilation schemes, suggest specific variants perform better based on ensemble sizes and model specifics ([Wiley Online Library](https://agupubs.onlinelibrary.wiley.com/doi/full/10.1029/2018WR023374)).

### Engineering and Systems Sciences

- **Two-time Scale EnKF for Nonlinear Systems**:
  A study on EnKF application to nonlinear, two-time scale systems, such as gas turbines, highlights its superior performance in computational efficiency and accuracy when compared to particle filtering methods. This effectiveness is measured using the equivalent flop (EF) complexity metric, showcasing EnKF's viability in engineering applications ([Daroogheh et al., 2017](http://arxiv.org/pdf/1710.05244v1)).

- **Pilot Study in Geostatistical Inversion**:
  Comparative studies employing PP-EnKF in geostatistical inversion demonstrate its effectiveness in scenarios with limited data, maintaining robust performance in terms of correlation structure accuracy and variance estimation compared to traditional approaches ([Keller et al., 2021](http://arxiv.org/pdf/2108.02164v1)).

### Financial and Other Domains

- **Binary State Vector Updates**:
  The EnKF concept was extended to update binary state vectors, maximizing unchanged component expectations, which proved effective over naive methods in test simulations relevant to petroleum reservoir dynamics. This demonstrated significant reduction in Frobenius norm, indicating better adherence to marginal filtering probabilities ([Loe & Tjelmeland, 2019](http://arxiv.org/pdf/1904.05107v1)).

## Conclusion

The Ensemble Kalman Filter remains a highly adaptable tool across numerous research disciplines, showing particular strength in environments needing computational efficiency and adaptability to high-dimensional data. When tailored to specific applications, such as meteorology, engineering, and financial systems, EnKF variants like KFHE and PP-EnKF address specific computational challenges and demonstrate enhanced performance in accuracy and computational resource management. Future studies could benefit from further integrating EnKF methodologies with machine learning techniques to enhance robustness and prediction accuracy in highly dynamic and complex systems.

## Check grading feedback

If you have collapsed the right panel to have more screen space for your code, as shown below:

<img src="./images/collapsed.png" alt="Collapsed Image" width="800" height="400"/>

You can click on the left-facing arrow button (highlighted in red) to view feedback for your submission after submitting it for grading. Once expanded, it should display like this:

<img src="./images/expanded.png" alt="Expanded Image" width="800" height="400"/>